## Window Functions 
- 테이블에서 행집합을 대상으로 하는 함수
- 집합 단위로 계산하기 때문에, 집계 함수와 비슷
- 단, 집계 함수는 한 행으로 결괏값을 보여주는 반면, 윈도우 암수는 각 행마다 처리 결과를 출력함
- 윈도우 함수를 사용하려면 집약함수 뒤에 OVER를 붙이고 윈도 함수를 지정합니다. 

### ROW Number()
- 각 행에 숫자를 1, 2, 3, ..., N 형태로 추가하도록 합니다.
- Row_N 기준으로 오름차순으로 진행합니다.  

```SQL
SELECT
  ROW_NUMBER() OVER() AS Row_N 
  , *
FROM Summer_Medals
ORDER BY Row_N ASC;
 row_n | year |  city  |  sport   | discipline |      athlete      | country | gender |           event            | medal  
-------+------+--------+----------+------------+-------------------+---------+--------+----------------------------+--------
     1 | 1896 | Athens | Aquatics | Swimming   | HAJOS Alfred      | HUN     | Men    | 100M Freestyle             | Gold
     2 | 1896 | Athens | Aquatics | Swimming   | HERSCHMANN Otto   | AUT     | Men    | 100M Freestyle             | Silver
     3 | 1896 | Athens | Aquatics | Swimming   | DRIVAS Dimitrios  | GRE     | Men    | 100M Freestyle For Sailors | Bronze
     4 | 1896 | Athens | Aquatics | Swimming   | MALOKINIS Ioannis | GRE     | Men    | 100M Freestyle For Sailors | Gold
     5 | 1896 | Athens | Aquatics | Swimming   | CHASAPIS Spiridon | GRE     | Men    | 100M Freestyle For Sailors | Silver
(5 rows)

```



- 이번에는 올림픽 년도를 오름차순 순번대로 작성을 하도록 합니다. 
- 이 때, 중요한 건 서브쿼리로 연도만을 추출한 뒤, 윈도우 함수를 이용한 것입니다. 

```SQL
SELECT 
    Year, 
    ROW_NUMBER() OVER() AS Row_N
FROM (
    SELECT DISTINCT Year
    FROM summer_medals
    ORDER BY Year ASC
) AS Years
ORDER BY Year ASC;
 year | row_n 
------+-------
 1896 |     1
 1900 |     2
 1904 |     3
 1908 |     4
 1912 |     5
(5 rows)

```



## ORDER BY
- 하계 올림픽이 열린 각 연도에 번호를 할당합니다. 
- 가장 최근 연도를 가진 행이 더 낮은 행 수를 갖도록 합니다.

```SQL
SELECT
  Year,
  -- Assign the lowest numbers to the most recent years
  ROW_NUMBER() OVER (ORDER BY Year DESC) AS Row_N
FROM (
  SELECT DISTINCT Year
  FROM Summer_Medals
) AS Years
ORDER BY Year;
```

- 

### ORDER BY
- 각 운동선수들이 획득한 메달 갯수를 내림차순으로 정렬하도록 합니다. 

```SQL
SELECT
  Athlete,
  COUNT(*) AS Medals
FROM Summer_Medals
GROUP BY Athlete
ORDER BY Medals DESC
LIMIT 5;
       athlete       | medals 
---------------------+--------
 PHELPS Michael      |     22
 LATYNINA Larisa     |     18
 ANDRIANOV Nikolay   |     15
 MANGIAROTTI Edoardo |     13
 SHAKHLIN Boris      |     13
(5 rows)

```
- 이전 쿼리에서, 각 선수들의 랭킹을 추가하도록 합니다. 

```SQL
WITH Athlete_Medals AS (
  SELECT
    -- Count the number of medals each athlete has earned
    Athlete,
    COUNT(*) AS Medals
  FROM Summer_Medals
  GROUP BY Athlete)

SELECT
  -- Number each athlete by how many medals they've earned
  Medals, 
  Athlete,
  ROW_NUMBER() OVER (ORDER BY Medals DESC) AS Row_N
FROM Athlete_Medals
ORDER BY Medals DESC
LIMIT 5;
 medals |       athlete       | row_n 
--------+---------------------+-------
     22 | PHELPS Michael      |     1
     18 | LATYNINA Larisa     |     2
     15 | ANDRIANOV Nikolay   |     3
     13 | MANGIAROTTI Edoardo |     4
     13 | ONO Takashi         |     5
(5 rows)

```

- 남자 69KG 역도 경기에서 매년 금메달리스트 조회하도록 합니다. 

```SQL
SELECT
    -- Return each year's champions' countries
    Year,
    Country AS champion
  FROM Summer_Medals
  WHERE
    Discipline = 'Weightlifting' AND
    Event = '69KG' AND
    Gender = 'Men' AND
    Medal = 'Gold';
 year | champion 
------+----------
 2000 | BUL   
 2004 | CHN   
 2008 | CHN   
 2012 | CHN   
(4 rows)
```

- 기존 쿼리에서 매년 전년도 챔피언도 같이 조회하도록 합니다. 

```SQL
WITH Weightlifting_Gold AS (
  SELECT
    -- Return each year's champions' countries
    Year,
    Country AS champion
  FROM Summer_Medals
  WHERE
    Discipline = 'Weightlifting' AND
    Event = '69KG' AND
    Gender = 'Men' AND
    Medal = 'Gold')

SELECT
  Year, Champion,
  -- Fetch the previous year's champion
  LAG(Champion) OVER
    (ORDER BY Year ASC) AS Last_Champion
FROM Weightlifting_Gold
ORDER BY Year ASC;
 year | champion | last_champion 
------+----------+---------------
 2000 | BUL      | 
 2004 | CHN      | BUL   
 2008 | CHN      | CHN   
 2012 | CHN      | CHN   
(4 rows)

```


## LAG() 함수의 활용
- Lag() 함수는 이전 행의 컬럼 값과 비교를 하거나 또는 값을 가져올 때 사용합니다. 
- 이 때 사용하는 문법이 LAG() OVER() 구문입니다. 

- 우선, 200M 달리기 금메달을 확인해보도록 한다. 

```SQL
SELECT 
	Year, 
	Athlete, 
	Country AS Champion
FROM summer_medals
WHERE 
	Discipline = 'Athletics' AND 
	Event = '200M' AND 
	Gender = 'Men' AND 
	Medal = 'Gold';

 year |           athlete            | champion 
------+------------------------------+----------
 1900 | TEWKSBURY Walter B. John     | USA
 1904 | HAHN Archie                  | USA
 1908 | KERR Robert                  | CAN
 1912 | CRAIG Ralph                  | USA
 1920 | WOODRING Allen               | USA
 1924 | SCHOLZ Jackson               | USA
 1928 | WILLIAMS Percy               | CAN
 1932 | TOLAN Eddie                  | USA
 1936 | OWENS Jesse                  | USA
 1948 | PATTON Melvin Emery          | USA
 1952 | STANFIELD Andrew William     | USA
 1956 | MORROW Robert Joseph         | USA
 1960 | BERRUTI Livio                | ITA
 1964 | CARR Henry                   | USA
 1968 | SMITH Thomas C.              | USA
 1972 | BORZOV Valery                | URS
 1976 | QUARRIE Donald               | JAM
 1980 | MENNEA Pietro                | ITA
 1984 | LEWIS Carl                   | USA
 1988 | DELOACH Joseph Nathaniel Jr. | USA
 1992 | MARSH Michael                | USA
 1996 | JOHNSON Michael              | USA
 2000 | KENTERIS Konstantinos        | GRE
 2004 | CRAWFORD Shawn               | USA
 2008 | BOLT Usain                   | JAM
 2012 | BOLT Usain                   | JAM
 
(26 rows)

```

- 그러나, 여기에서 한걸음 더 나아가, 2연패를 달성한 선수가 있는지 확인을 해보도록 한다. 

```SQL
WITH Gold AS (
SELECT 
	Year, 
	Athlete AS Champion
FROM summer_medals
WHERE 
	Discipline = 'Athletics' AND 
	Event = '200M' AND 
	Gender = 'Men' AND 
	Medal = 'Gold')

SELECT 
	Year, Champion,
	LAG(Champion) OVER (ORDER BY YEAR ASC) AS Last_Champion
FROM Gold
ORDER BY Year ASC;
 year |           champion           |        last_champion         
------+------------------------------+------------------------------
 1900 | TEWKSBURY Walter B. John     | 
 1904 | HAHN Archie                  | TEWKSBURY Walter B. John
 1908 | KERR Robert                  | HAHN Archie
 1912 | CRAIG Ralph                  | KERR Robert
 1920 | WOODRING Allen               | CRAIG Ralph
 1924 | SCHOLZ Jackson               | WOODRING Allen
 1928 | WILLIAMS Percy               | SCHOLZ Jackson
 1932 | TOLAN Eddie                  | WILLIAMS Percy
 1936 | OWENS Jesse                  | TOLAN Eddie
 1948 | PATTON Melvin Emery          | OWENS Jesse
 1952 | STANFIELD Andrew William     | PATTON Melvin Emery
 1956 | MORROW Robert Joseph         | STANFIELD Andrew William
 1960 | BERRUTI Livio                | MORROW Robert Joseph
 1964 | CARR Henry                   | BERRUTI Livio
 1968 | SMITH Thomas C.              | CARR Henry
 1972 | BORZOV Valery                | SMITH Thomas C.
 1976 | QUARRIE Donald               | BORZOV Valery
 1980 | MENNEA Pietro                | QUARRIE Donald
 1984 | LEWIS Carl                   | MENNEA Pietro
 1988 | DELOACH Joseph Nathaniel Jr. | LEWIS Carl
 1992 | MARSH Michael                | DELOACH Joseph Nathaniel Jr.
 1996 | JOHNSON Michael              | MARSH Michael
 2000 | KENTERIS Konstantinos        | JOHNSON Michael
 2004 | CRAWFORD Shawn               | KENTERIS Konstantinos
 2008 | BOLT Usain                   | CRAWFORD Shawn
 2012 | BOLT Usain                   | BOLT Usain
(26 rows)

```

## Partition by
- Partition BY는 열의 고유 값을 기준으로 테이블을 파티션으로 분할함. 
    - 결과가 한 열로 롤업되지 않음 
- 창 기능에 따라 별도로 작동됨
    - ROW_NUMBER가 각 파티션에 대해 재설정됩니다.
    - LAG는 이전 행이 동일한 파티션에 있는 경우에만 이전 값을 가져옵니다.
- 이름 Gender, Year, Country가 표시되도록 쿼리를 작성한다. 

```SQL
SELECT DISTINCT
    Athlete, Gender, Year, Country
FROM Summer_Medals
WHERE
	Year >= 1992 AND
    Event = '200M' AND
    Medal = 'Gold' 
ORDER BY Year;
         athlete         | gender | year | country
-------------------------+--------+------+---------
 MARSH Michael           | Men    | 1992 | USA
 TORRENCE Gwen           | Women  | 1992 | USA
 JOHNSON Michael         | Men    | 1996 | USA
 PEREC Marie-Jose        | Women  | 1996 | FRA
 DAVIS Pauline Elaine    | Women  | 2000 | BAH
 KENTERIS Konstantinos   | Men    | 2000 | GRE
 CAMPBELL-BROWN Veronica | Women  | 2004 | JAM
 CRAWFORD Shawn          | Men    | 2004 | USA
 BOLT Usain              | Men    | 2008 | JAM
 CAMPBELL-BROWN Veronica | Women  | 2008 | JAM
 BOLT Usain              | Men    | 2012 | JAM
 FELIX Allyson           | Women  | 2012 | USA
(12개 행)

```

- 위 쿼리를 근거로 하여 Partition By 문법을 적용하도록 합니다. 

```SQL
With Gold AS (
SELECT DISTINCT
    Gender, Year, Country
FROM Summer_Medals
WHERE
	Year >= 1992 AND
    Event = '200M' AND
    Medal = 'Gold'
)

SELECT 
	Gender, Year, Country AS Currnet_Champion, 
	LAG(Country) OVER (PARTITION BY Gender ORDER BY Year ASC) AS Previous_Champion
FROM Gold
ORDER BY Gender ASC, Year ASC;

 gender | year | currnet_champion | previous_champion
--------+------+------------------+-------------------
 Men    | 1992 | USA              |
 Men    | 1996 | USA              | USA
 Men    | 2000 | GRE              | USA
 Men    | 2004 | USA              | GRE
 Men    | 2008 | JAM              | USA
 Men    | 2012 | JAM              | JAM
 Women  | 1992 | USA              |
 Women  | 1996 | FRA              | USA
 Women  | 2000 | BAH              | FRA
 Women  | 2004 | JAM              | BAH
 Women  | 2008 | JAM              | JAM
 Women  | 2012 | USA              | JAM
(12개 행)

```

- 이번에는 경기종목을 2개로 추가하도록 합니다. ('200M', '10000M')
    + 이 때에는 IN() 함수를 사용합니다. 
    
```SQL
With Gold AS (
SELECT DISTINCT
    Gender, Year, Event, Country
FROM Summer_Medals
WHERE
	Year >= 1992 AND
    Event IN('200M', '10000M') AND
    Medal = 'Gold'
)

SELECT
  Gender, Year, Event,
  Country AS Champion,
  LAG(Country) OVER (PARTITION BY Gender, Event
                         ORDER BY Year ASC) AS Last_Champion
FROM Gold
ORDER BY Event ASC, Gender ASC, Year ASC;

 gender | year | event  | champion | last_champion
--------+------+--------+----------+---------------
 Men    | 1992 | 10000M | MAR      |
 Men    | 1996 | 10000M | ETH      | MAR
 Men    | 2000 | 10000M | ETH      | ETH
 Men    | 2004 | 10000M | ETH      | ETH
 Men    | 2008 | 10000M | ETH      | ETH
 Men    | 2012 | 10000M | GBR      | ETH
 Women  | 1992 | 10000M | ETH      |
 Women  | 1996 | 10000M | POR      | ETH
 Women  | 2000 | 10000M | ETH      | POR
 Women  | 2004 | 10000M | CHN      | ETH
 Women  | 2008 | 10000M | ETH      | CHN
 Women  | 2012 | 10000M | ETH      | ETH
 Men    | 1992 | 200M   | USA      |
 Men    | 1996 | 200M   | USA      | USA
 Men    | 2000 | 200M   | GRE      | USA
 Men    | 2004 | 200M   | USA      | GRE
 Men    | 2008 | 200M   | JAM      | USA
 Men    | 2012 | 200M   | JAM      | JAM
 Women  | 1992 | 200M   | USA      |
 Women  | 1996 | 200M   | FRA      | USA
 Women  | 2000 | 200M   | BAH      | FRA
 Women  | 2004 | 200M   | JAM      | BAH
 Women  | 2008 | 200M   | JAM      | JAM
 Women  | 2012 | 200M   | USA      | JAM
(24개 행)

```